In [7]:
import numpy as np
import qiskit as qk
import matplotlib.pyplot as plt
from qiskit import Aer
from tqdm.notebook import tqdm

import sys
sys.path.insert(0, '../../src/')
from neuralnetwork import *
from analysis import *
from data_encoders import *
from parametrizations import *

#%matplotlib notebook
%matplotlib inline
%load_ext autoreload
%autoreload 2

np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
backend = Aer.get_backend('qasm_simulator')
n_samples = 2
n_features = 2
n_targets = 1

x = np.linspace(0, 2, n_samples)
y = x**2
y = y - np.min(y)
y = y/np.max(y)
y = np.pi *y
y = y.reshape(-1,1)

X = np.zeros((n_samples, n_features))
X[:,0] = x
X[:,1] = x**2
#X[:,2] = x**3
#X[:,3] = x**4



X = X - np.min(X, axis=0).reshape(1,-1)
X = X/np.max(X, axis=0).reshape(1,-1)
X = np.pi * X

print(X)
print(y)

[[0.000 0.000]
 [3.142 3.142]]
[[0.000]
 [3.142]]


In [20]:
reps = 4
np.random.seed(42)
model = ParallelModel(n_features=n_features,
                      n_targets=n_targets,
                      reps=reps,
                      backend=backend, 
                      shots=100000)

optimizer = Adam()
optimizer.initialize(model.theta.shape)

In [21]:
for i in tqdm(range(100)):
    grad = [model.gradient(X, y)]
    grad = optimizer(grad)
    model.theta += -0.01*grad[0]
    y_pred = model.predict(X)
    mse = np.mean((y_pred - y)**2)
    loss = model.loss(X,y)
    #print(model.theta)
    print(f"{loss=:.3f}, {mse=:.3f}, {y_pred.flatten()=}")

  0%|          | 0/100 [00:00<?, ?it/s]

loss=0.541, mse=0.349, y_pred.flatten()=array([0.394, 2.405])
loss=0.535, mse=0.316, y_pred.flatten()=array([0.380, 2.444])
loss=0.534, mse=0.283, y_pred.flatten()=array([0.355, 2.479])
loss=0.529, mse=0.253, y_pred.flatten()=array([0.350, 2.522])
loss=0.535, mse=0.217, y_pred.flatten()=array([0.326, 2.568])
loss=0.522, mse=0.195, y_pred.flatten()=array([0.309, 2.599])
loss=0.518, mse=0.165, y_pred.flatten()=array([0.290, 2.645])
loss=0.518, mse=0.142, y_pred.flatten()=array([0.267, 2.680])
loss=0.519, mse=0.121, y_pred.flatten()=array([0.251, 2.719])
loss=0.514, mse=0.100, y_pred.flatten()=array([0.235, 2.762])
loss=0.511, mse=0.084, y_pred.flatten()=array([0.221, 2.797])
loss=0.511, mse=0.069, y_pred.flatten()=array([0.197, 2.826])
loss=0.504, mse=0.055, y_pred.flatten()=array([0.180, 2.863])
loss=0.507, mse=0.042, y_pred.flatten()=array([0.161, 2.901])
loss=0.504, mse=0.032, y_pred.flatten()=array([0.143, 2.934])
loss=0.500, mse=0.022, y_pred.flatten()=array([0.124, 2.974])
loss=0.5

KeyboardInterrupt: 

In [ ]:
data = np.array([[0,0], [1,1], [2,2], [3,3]])
encoder = ParallelEncoder()


features = qk.QuantumRegister(2, name="features")
ancilla = qk.QuantumRegister(2, name="ancilla_f")
registers = [features, ancilla]
circuit = qk.QuantumCircuit(*registers)

circuit = encoder(circuit, features, ancilla, data)

circuit.draw()